In [17]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

In [18]:
data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [19]:
############################################################################
# Req 3-1: Padding 함수 구현하기                                             #
############################################################################

def padding(data, pad_value=0):
    """
    Args:
        data (list[list[int]]): data sequence
        pad_value (int): value to pad
    Returns:
        data (list[list[int]]): padded data sequence
        max_len (int): maximum length of intput data
    """
    ################################################################################
    # TODO: 배치 내 데이터 중 최대 길이에 맞추어 다른 데이터의 뒷부분에 패딩 값을 붙여        #
    # 길이를 동일하게 맞추어 반환함                                                     #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    # 1) 가장 긴 데이터의 길이 추출
    # 2) 모든 데이터를 돌면서 1에서 구한 길이에 맞게 뒷부분을 pad_value로 채우기

    # 1) 가장 긴 데이터의 길이 추출
    max_len = max(len(seq) for seq in data)

    # 2) 모든 데이터를 돌면서 1에서 구한 길이에 맞게 뒷부분을 pad_value로 채우기
    data2 = []

    for seq in data:
        # 현재 시퀀스의 길이가 max_len보다 짧으면 pad_value로 패딩을 추가
        if len(seq) < max_len:
            seq = seq + [pad_value] * (max_len - len(seq))
        data2.append(seq)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################

    return data2, max_len

In [20]:
pad_value = 0 # 패딩(Padding)은 제로(zero) 패딩으로, 빈 공간을 0으로 채움.
data, max_len = padding(data)
print(f"Maximum sequence length: {max_len}")
for d in data:
    print(d)

Maximum sequence length: 20
[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0]
[60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10]
[70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0]
[20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]


In [21]:
# 한 데이터의 최대 길이(vocab_size)는 100이라 가정.
vocab_size = 100

# hidden size of model
d_k = 64

In [22]:
############################################################################
# Req 3-2: Embedding 및 Query, Key, Value Vector 구하기                     #
############################################################################

################################################################################
# TODO: Embedding 레이어를 정의하고, 입력 데이터를 임베딩화함                         #
# 또한 Q, K, V를 위한 레이어를 정의하고, 임베딩 벡터를 각각 Q, K, V 벡터로 변환함       #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) vocab_size 개수의 데이터 각각을 d_k 차원으로 표현할 임베딩을 할 레이어 정의
# 2) 임베딩 레이어를 사용하여 입력 데이터를 임베딩 벡터로 projection
# 3) d_k 차원의 Q, K, V 벡터를 만들 레이어 정의
# 4) 임베딩 벡터를 Q, K, V 벡터로 변환

embedding_dim = d_k  # 임베딩 벡터의 차원, Q, K, V와 동일하게 설정

# 1) vocab_size 개수의 데이터 각각을 d_k 차원으로 표현할 임베딩 레이어 정의
embedding = nn.Embedding(vocab_size, embedding_dim)

# 2) 임베딩 레이어를 사용하여 입력 데이터를 임베딩 벡터로 projection
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_k)
print(f"Shape of embedding: {batch_emb.shape}")

# 3) d_k 차원의 Q, K, V 벡터를 만들 레이어 정의
w_q = nn.Linear(embedding_dim, d_k)
w_k = nn.Linear(embedding_dim, d_k)
w_v = nn.Linear(embedding_dim, d_k)

# 4) 임베딩 벡터를 Q, K, V 벡터로 변환
q = w_q(batch_emb)  # (B, L, d_k)
k = w_k(batch_emb)  # (B, L, d_k)
v = w_v(batch_emb)  # (B, L, d_k)
print(f"Shape of Q, K, V: {q.shape, k.shape, v.shape}")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Shape of embedding: torch.Size([10, 20, 64])
Shape of Q, K, V: (torch.Size([10, 20, 64]), torch.Size([10, 20, 64]), torch.Size([10, 20, 64]))


In [23]:
############################################################################
# Req 3-3: Scaled Dot-Product Self-Attention 구현하기                        #
############################################################################

################################################################################
# TODO: 수식에 맞추어 scaled dot-product self-attention을 구현함                   #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# 1) Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근으로 나눔 (=(Q x K^T) / sqrt(d_k))
# Output shape: (B, L, L)
attn_scores = torch.bmm(q, k.transpose(1, 2)) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))
print(f"Shape of attn_scores: {attn_scores.shape}")

# 2) 위 값에 softmax를 취함. row-wise이기 때문에 dim은 -1 로 적용할 것.
# Output shape: (B, L, L)
attn_dists = F.softmax(attn_scores, dim=-1)
print(f"Shape of attn_dists: {attn_dists.shape}")

# 3) Value 벡터를 곱해 최종 attention value 계산
# Output shape: (B, L, d_k)
attn_values = torch.bmm(attn_dists, v)
print(f"Shape of attn_values: {attn_values.shape}")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Shape of attn_scores: torch.Size([10, 20, 20])
Shape of attn_dists: torch.Size([10, 20, 20])
Shape of attn_values: torch.Size([10, 20, 64])


In [24]:
# 한 데이터의 최대 길이(vocab_size)는 100이라 가정.
vocab_size = 100

# hidden size of model
d_k = 64

# hidden size of model
d_model = 512

# number of attention heads
num_heads = 8

In [25]:
############################################################################
# Req 3-4: Embedding 및 Query, Key, Value Vector 구하기                      #
############################################################################

################################################################################
# TODO: Embedding 레이어를 정의하고, 입력 데이터를 임베딩화함                         #
# 또한 Q, K, V를 위한 레이어를 정의하고, 임베딩 벡터를 각각 Q, K, V 벡터로 변환함       #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# 1) vocab_size 개수의 데이터 각각을 d_model 차원으로 표현할 임베딩을 할 레이어 정의
embedding = nn.Embedding(vocab_size, d_model)

# 2) 임베딩 레이어를 사용하여 입력 데이터를 임베딩 벡터로 projection
batch = torch.LongTensor(data)  # (B, L)
batch_size = batch.shape[0]
batch_emb = embedding(batch)  # (B, L, d_model)
print(f"Shape of embedding: {batch_emb.shape}")

# 3) d_model 차원의 Q, K, V 벡터를 만들 레이어 정의
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

# 4) 임베딩 벡터를 Q, K, V 벡터로 변환
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)
print(f"Shape of Q, K, V: {q.shape, k.shape, v.shape}")

# 5) d_k 차원 계산
# 이미 d_k = d_model // num_heads로 설정되어 있음

# 6) Q, K, V 벡터를 각 헤드만큼 쪼갬 (각 헤드의 각 벡터 차원은 d_k)
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

# 7) Self-attention 연산을 위하여 각 헤드가 (L, d_k) 행렬을 갖도록 축을 transpose
q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)
print(f"Shape of Q, K, V: {q.shape, k.shape, v.shape}")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Shape of embedding: torch.Size([10, 20, 512])
Shape of Q, K, V: (torch.Size([10, 20, 512]), torch.Size([10, 20, 512]), torch.Size([10, 20, 512]))
Shape of Q, K, V: (torch.Size([10, 8, 20, 64]), torch.Size([10, 8, 20, 64]), torch.Size([10, 8, 20, 64]))


In [26]:
############################################################################
# Req 3-5: Scaled Dot-Product Self-Attention 구현하기                        #
############################################################################

################################################################################
# TODO: 수식에 맞추어 scaled dot-product self-attention을 구현함                   #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# 1) Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근으로 나눔 (=(Q x K^T) / sqrt(d_k))
# Output shape - (B, num_heads, L, L)
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))
print(f"Shape of attn_scores: {attn_scores.shape}")

# 2) 위 값에 softmax를 취함. row-wise이기 때문에 dim은 -1 로 적용할 것.
# Output shape: (B, num_heads, L, L)
attn_dists = F.softmax(attn_scores, dim=-1)
print(f"Shape of attn_dists: {attn_dists.shape}")

# 3) Value 벡터를 곱해 최종 attention value 계산
# Output shape: (B, num_heads, L, d_k)
attn_values = torch.matmul(attn_dists, v)
print(f"Shape of attn_values: {attn_values.shape}")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Shape of attn_scores: torch.Size([10, 8, 20, 20])
Shape of attn_dists: torch.Size([10, 8, 20, 20])
Shape of attn_values: torch.Size([10, 8, 20, 64])


In [27]:
############################################################################
# Req 3-6: Attention heads의 결과물 병합하기                                  #
############################################################################

################################################################################
# TODO: 각 attention head의 결과물을 병합하고 head 별 가중치로 projection하여        #
# 최종 attention 결과를 구함                                                     #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# 1) Scaled dot-product 의 결과(attn_values)를 (B, L, num_heads, d_k) 형태로 축 변경
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
print(f"Shape of attn_values after transpose: {attn_values.shape}")

# 2) 각 head의 점수를 concatenate함
attn_values = attn_values.reshape(batch_size, -1, d_model)  # (B, L, d_model)
print(f"Shape of attention values from all heads: {attn_values.shape}")

# 3) 각 head마다의 가중치를 부여할 linear layer 정의
w_0 = nn.Linear(d_model, d_model)

# 4) 위 linear projection layer를 통과하여 최종 출력을 계산
outputs = w_0(attn_values)  # (B, L, d_model)
print(f"Shape of multi-head self-attention result: {outputs.shape}")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Shape of attn_values after transpose: torch.Size([10, 20, 8, 64])
Shape of attention values from all heads: torch.Size([10, 20, 512])
Shape of multi-head self-attention result: torch.Size([10, 20, 512])


In [28]:
############################################################################
# Req 3-7: Multi-Head Self-Attention의 모듈 클래스 구현                       #
############################################################################

class MultiheadAttention(nn.Module):
    def __init__(self, d_model, d_k, num_heads):
        super(MultiheadAttention, self).__init__()

        self.d_model = d_model
        self.d_k = d_k
        self.num_heads = num_heads

        # 1) 임베딩 벡터를 Q, K, V 벡터로 변환할 레이어 정의
        # Each of these layers maps d_model to d_k
        self.w_q = nn.Linear(d_model, d_k * num_heads)
        self.w_k = nn.Linear(d_model, d_k * num_heads)
        self.w_v = nn.Linear(d_model, d_k * num_heads)

        # 2) 각 헤드의 결과에 가중치를 부여할 linear layer 정의
        self.w_o = nn.Linear(d_k * num_heads, d_model)

    def self_attention(self, q, k, v):
        """scaled-dot product attention

        Args:
            q, k, v (torch.Tensor): Query, Key, Value vectors
        Returns:
            attn_values (torch.Tensor): Result of self-attention
        """
        # 1) Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근으로 나눔 (=(Q x K^T) / sqrt(d_k))
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))

        # 2) 위 값에 softmax를 취함. row-wise이기 때문에 dim은 -1 로 적용할 것.
        attn_dists = F.softmax(attn_scores, dim=-1)

        # 3) Value 벡터를 곱해 최종 attention value 계산
        attn_values = torch.matmul(attn_dists, v)

        return attn_values

    def forward(self, batch_emb):
        batch_size = batch_emb.shape[0]
        seq_len = batch_emb.shape[1]

        # 1) 입력 데이터를 Q, K, V 벡터로 변환
        q = self.w_q(batch_emb)  # (B, L, d_k * num_heads)
        k = self.w_k(batch_emb)  # (B, L, d_k * num_heads)
        v = self.w_v(batch_emb)  # (B, L, d_k * num_heads)

        # 2) 위 결과를 Head의 수로 분할함
        q = q.view(batch_size, seq_len, self.num_heads, self.d_k)  # (B, L, num_heads, d_k)
        k = k.view(batch_size, seq_len, self.num_heads, self.d_k)  # (B, L, num_heads, d_k)
        v = v.view(batch_size, seq_len, self.num_heads, self.d_k)  # (B, L, num_heads, d_k)

        # 3) 각 head가 (L, d_k)의 matrix를 담당하도록 만듦
        q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
        k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
        v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

        # 4) Scaled dot-product self-attention 연산을 수행함
        attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)

        # 5) 각 attention head의 결과물을 concatenate해 병합함
        attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
        attn_values = attn_values.contiguous().view(batch_size, seq_len, self.d_model)  # (B, L, d_model)

        # 6) head 별로 정해진 가중치로 linear projection하여 최종 출력을 결정함
        outputs = self.w_o(attn_values)  # (B, L, d_model)

        return outputs

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

In [29]:
multi_attn = MultiheadAttention(d_model, d_k, num_heads)
outputs = multi_attn(batch_emb)  # (B, L, d_model)

print(outputs)
print(outputs.shape)

tensor([[[-0.1000,  0.0929,  0.1080,  ..., -0.0644,  0.0509,  0.1925],
         [-0.0399,  0.0436,  0.1389,  ..., -0.0301,  0.0496,  0.1690],
         [-0.0580,  0.0631,  0.1083,  ...,  0.0279,  0.1052,  0.1502],
         ...,
         [-0.0502,  0.1020,  0.0912,  ..., -0.0265,  0.0589,  0.1189],
         [-0.0502,  0.1020,  0.0912,  ..., -0.0265,  0.0589,  0.1189],
         [-0.0502,  0.1020,  0.0912,  ..., -0.0265,  0.0589,  0.1189]],

        [[-0.2198,  0.1510, -0.1137,  ..., -0.3374,  0.2957,  0.4713],
         [-0.1906,  0.1591, -0.0483,  ..., -0.3505,  0.3057,  0.4240],
         [-0.2532,  0.1640, -0.0428,  ..., -0.3516,  0.3230,  0.4309],
         ...,
         [-0.1612,  0.1964, -0.0998,  ..., -0.2925,  0.2985,  0.4221],
         [-0.1612,  0.1964, -0.0998,  ..., -0.2925,  0.2985,  0.4221],
         [-0.1612,  0.1964, -0.0998,  ..., -0.2925,  0.2985,  0.4221]],

        [[-0.1423,  0.0851,  0.0617,  ..., -0.0927,  0.0178,  0.2986],
         [-0.0787,  0.1375,  0.0315,  ..., -0